# CNN

Here I use [Pytorch](https://pytorch.org) as the NN framework

In [1]:
import torch
import os
import torchvision.transforms as T
from FDDB_dataloader import FDDB
from torch.utils.data import DataLoader
import tqdm
import torchvision.models as models
import torch.nn as nn
import torch.optim
import torch.optim.lr_scheduler as lr_scheduler
import time
import shutil

## DataLoader

### Channel stat values

In [2]:
transforms = T.Compose([T.Resize((96,96)), T.ToTensor()])

In [8]:
db_train = FDDB('train_list.npy', transform=transforms)
db_val = FDDB('val_list.npy', transform=transforms)

In [12]:
db_train[0][0].mean([1,2])

tensor([0.6139, 0.4767, 0.4134])

In [18]:
mean = torch.zeros(3)
std = torch.zeros(3)

In [19]:
for img, label in tqdm.tqdm(db_train):
    mean += img.mean([1,2])
mean /= len(db_train)
mean

100%|██████████| 20680/20680 [00:25<00:00, 800.96it/s]


tensor([0.5116, 0.4200, 0.3651])

In [20]:
tmp = mean.reshape([3,1,1])

for img, label in tqdm.tqdm(db_train):
    std += ((img - tmp)**2).mean([1,2])
std /= len(db_train)
std = std.sqrt()
std

100%|██████████| 20680/20680 [00:26<00:00, 784.72it/s]


tensor([0.2655, 0.2430, 0.2420])

In [22]:
del db_train, db_val

## Parameters:

In [5]:
batch_size = 256
workers = 4
gpu = 1
arch = 'resnet18'
optim = 'Adam'
lr = 0.1
step_size = 30
gamma = 0.1
weight_decay = 1e-4
momentum = 0.9
epochs = 200
save_epoch = 50
save = False
suffix = ''
print_freq = 10

## Create Pytorch MultiThread DataLoader

In [6]:
normalize = T.Normalize(mean = [0.5116, 0.4200, 0.3651],
                         std = [0.2655, 0.2430, 0.2420])

In [7]:
db_train = FDDB('train_list.npy', 
                transform=T.Compose([
                    T.Resize((96,96)),
                    T.RandomHorizontalFlip(),
                    T.ToTensor(),
                    normalize
                ]),
                zero_one=True
               )
db_val = FDDB('val_list.npy', 
              transform=T.Compose([
                  T.Resize((96,96)),
                  T.ToTensor(),
                  normalize
              ]),
              zero_one=True
             )

In [8]:
train_loader = DataLoader(db_train, batch_size=batch_size, shuffle=True,
                         num_workers=workers, pin_memory=True)
val_loader = DataLoader(db_val, batch_size=batch_size, shuffle=False,
                       num_workers=workers, pin_memory=True)

## Create CNN

In [10]:
cnn = models.__dict__[arch](num_classes=1)
criterion = nn.BCEWithLogitsLoss()
if optim == 'SGD':
    optimizer = torch.optim.SGD(cnn.parameters(), lr, momentum = momentum, weight_decay=weight_decay)
elif optim == 'Adam':
    optimizer = torch.optim.Adam(cnn.parameters(), lr, weight_decay=weight_decay)


In [11]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top0 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if gpu is not None:
            input = input.cuda(gpu, non_blocking=True)
            target = target.float().unsqueeze(-1).cuda(gpu, non_blocking=True)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        acc = accuracy_VOC2012(output, target)
        
        losses.update(loss.item(), input.size(0))
        top0.update(acc[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Acc@0 {top0.val:.3f} ({top0.avg:.3f})\t'.format(
                epoch, i, len(train_loader), batch_time=batch_time,
                data_time=data_time, loss=losses, top0=top0))



In [12]:
def validate(val_loader, model, criterion, epoch):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top0 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            if gpu is not None:
                input = input.cuda(gpu, non_blocking=True)
                target = target.float().unsqueeze(-1).cuda(gpu, non_blocking=True)

            # compute output
            output = model(input)
            loss = criterion(output, target)

            acc = accuracy_VOC2012(output, target)
            losses.update(loss.item(), input.size(0))
            top0.update(acc[0], input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
            if i % print_freq == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Acc@0 {top0.val:.3f} ({top0.avg:.3f})\t'.format(
                    i, len(val_loader), batch_time=batch_time, loss=losses,
                    top0=top0))

        print(' * Acc@0 {top0.avg:.3f}'.format(top0=top0))
    return top0.avg

In [13]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best_{}_{}.pth.tar'.format(arch, suffix))


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def accuracy_VOC2012(output, target):
    with torch.no_grad():
        batch_size = target.size(0)
        accur = output.gt(0.).long().eq(target.long()).float().mean()
        res = []
        res.append(accur)
        return res

In [14]:
cnn.cuda(gpu)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [ ]:
best_acc1 = 0
scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
for epoch in range(epochs):
    # train for one epoch
    train(train_loader, cnn, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, cnn, criterion, epoch)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)
    if save:
        save_dir = 'checkpoint_{}_{}.pth.tar'.format(arch, suffix)
        if (epoch+1)%save_epoch == 0:
            save_checkpoint({
                'epoch': epoch + 1,
                'arch': arch,
                'state_dict': model.state_dict(),
                'best_acc1': best_acc1,
                'acc1': acc1,
                'optimizer' : optimizer.state_dict(),
            }, is_best, save_dir)
    scheduler.step()

Epoch: [0][0/81]	Time 4.928 (4.928)	Data 4.185 (4.185)	Loss 0.6603 (0.6603)	Acc@0 0.703 (0.703)	
Epoch: [0][10/81]	Time 0.332 (0.749)	Data 0.000 (0.403)	Loss 0.5184 (0.9072)	Acc@0 0.789 (0.793)	
Epoch: [0][20/81]	Time 0.340 (0.553)	Data 0.000 (0.221)	Loss 0.4189 (0.7607)	Acc@0 0.773 (0.791)	
Epoch: [0][30/81]	Time 0.331 (0.482)	Data 0.000 (0.156)	Loss 0.3741 (0.6621)	Acc@0 0.828 (0.796)	
Epoch: [0][40/81]	Time 0.331 (0.446)	Data 0.000 (0.123)	Loss 0.4104 (0.6058)	Acc@0 0.820 (0.798)	
Epoch: [0][50/81]	Time 0.344 (0.424)	Data 0.000 (0.103)	Loss 0.4514 (0.5787)	Acc@0 0.770 (0.799)	
Epoch: [0][60/81]	Time 0.334 (0.410)	Data 0.000 (0.089)	Loss 0.3978 (0.5466)	Acc@0 0.832 (0.806)	
Epoch: [0][70/81]	Time 0.330 (0.399)	Data 0.000 (0.080)	Loss 0.3615 (0.5234)	Acc@0 0.867 (0.814)	
Epoch: [0][80/81]	Time 0.311 (0.391)	Data 0.000 (0.072)	Loss 0.2387 (0.4952)	Acc@0 0.930 (0.825)	
Test: [0/37]	Time 1.332 (1.332)	Loss 0.5973 (0.5973)	Acc@0 0.668 (0.668)	
Test: [10/37]	Time 0.108 (0.279)	Loss 0.6848 

Test: [20/37]	Time 0.308 (0.230)	Loss 0.0392 (0.3865)	Acc@0 0.992 (0.864)	
Test: [30/37]	Time 0.236 (0.218)	Loss 0.0741 (0.2765)	Acc@0 0.973 (0.905)	
 * Acc@0 0.919
Epoch: [7][0/81]	Time 1.583 (1.583)	Data 1.455 (1.455)	Loss 0.1454 (0.1454)	Acc@0 0.941 (0.941)	
Epoch: [7][10/81]	Time 0.339 (0.454)	Data 0.000 (0.151)	Loss 0.0894 (0.1331)	Acc@0 0.961 (0.952)	
Epoch: [7][20/81]	Time 0.346 (0.400)	Data 0.000 (0.089)	Loss 0.1294 (0.1289)	Acc@0 0.949 (0.953)	
Epoch: [7][30/81]	Time 0.332 (0.381)	Data 0.000 (0.067)	Loss 0.1148 (0.1196)	Acc@0 0.965 (0.957)	
Epoch: [7][40/81]	Time 0.339 (0.371)	Data 0.000 (0.056)	Loss 0.1384 (0.1221)	Acc@0 0.945 (0.956)	
Epoch: [7][50/81]	Time 0.338 (0.366)	Data 0.000 (0.049)	Loss 0.0843 (0.1173)	Acc@0 0.977 (0.958)	


# Packaging

In [27]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def accuracy_VOC2012(output, target):
    with torch.no_grad():
        batch_size = target.size(0)
        accur = output.gt(0.).long().eq(target.long()).float().mean()
        res = []
        res.append(accur)
        return res

class CNN():
    def __init__(self, params = {}):
        self.batch_size = self.set_params(256, 'batch_size', params)
        self.workers = self.set_params(4, 'workers', params)
        self.gpu = self.set_params(0, 'gpu', params)
        self.arch = self.set_params('resnet18', 'arch', params)
        self.optim = self.set_params('Adam', 'optim', params)
        self.lr = self.set_params(0.1, 'lr', params)
        self.step_size = self.set_params(30, 'step_size', params)
        self.gamma = self.set_params(0.1, 'gamma', params)
        self.weight_decay = self.set_params(1e-4, 'weight_decay', params)
        self.momentum = self.set_params(0.9, 'momentum', params)
        self.epochs = self.set_params(90, 'epochs', params)
        self.save_epoch = self.set_params(50, 'save_epoch', params)
        self.save = self.set_params(False, 'save', params)
        self.suffix = self.set_params('', 'suffix', params)
        self.print_freq = self.set_params(10, 'print_freq', params)
        
        self.cnn = models.__dict__[self.arch](num_classes=1)
        self.criterion = nn.BCEWithLogitsLoss()
        
        if self.gpu is not None:
            self.cnn.cuda(self.gpu)
            self.criterion.cuda(self.gpu)        
        
        if self.optim == 'SGD':
            self.optimizer = torch.optim.SGD(self.cnn.parameters(), self.lr, 
                                             momentum = self.momentum,
                                             weight_decay=self.weight_decay)
        elif self.optim == 'Adam':
            self.optimizer = torch.optim.Adam(self.cnn.parameters(), self.lr, 
                                              weight_decay = self.weight_decay)
        
        normalize = T.Normalize(mean = [0.5116, 0.4200, 0.3651],
                         std = [0.2655, 0.2430, 0.2420])
        self.img_trans = T.Compose([
                      T.Resize((96,96)),
                      T.ToTensor(),
                      normalize
                  ])
        
    def set_params(self, default, label, params):
        return default if label not in params.keys() else params[label]
    
    def train(self, epoch):
        batch_time = AverageMeter()
        data_time = AverageMeter()
        losses = AverageMeter()
        top0 = AverageMeter()

        # switch to train mode
        self.cnn.train()
        
#         enum = tqdm.tqdm(enumerate(self.train_loader),
#                          total= len(self.train_loader),
#                         desc='Train: Loss {loss.val:.4f} ({loss.avg:.4f}) '
#                              'Acc@0 {top0.val:.3f} ({top0.avg:.3f})'
#                          .format(loss=losses, top0=top0))
        
        for i, (input, target) in enumerate(self.train_loader):

            if self.gpu is not None:
                input = input.cuda(self.gpu, non_blocking=True)
                target = target.float().unsqueeze(-1).cuda(self.gpu, non_blocking=True)

            # compute output
            output = self.cnn(input)
            loss = self.criterion(output, target)

            acc = accuracy_VOC2012(output, target)

            losses.update(loss.item(), input.size(0))
            top0.update(acc[0], input.size(0))

            # compute gradient and do SGD step
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
#             if (i+1) % self.print_freq == 0:
#                 enum.set_description('Train: Loss {loss.val:.4f} ({loss.avg:.4f})\t'
#                                      'Acc@0 {top0.val:.3f} ({top0.avg:.3f})'
#                                      .format(loss=losses, top0=top0))
        return losses.avg, top0.avg

    def validate(self):
        batch_time = AverageMeter()
        losses = AverageMeter()
        top0 = AverageMeter()

        # switch to evaluate mode
        self.cnn.eval()

        with torch.no_grad():
#             enum = tqdm.tqdm(enumerate(self.val_loader), 
#                       total=len(self.val_loader), 
#                       desc='Test: Loss {loss.val:.4f} ({loss.avg:.4f}) '
#                           'Acc@0 {top0.val:.3f} ({top0.avg:.3f})'
#                       .format( loss=losses, top0=top0))
            for i, (input, target) in enumerate(self.val_loader):
                if self.gpu is not None:
                    input = input.cuda(self.gpu, non_blocking=True)
                    target = target.float().unsqueeze(-1).cuda(self.gpu, non_blocking=True)

                # compute output
                output = self.cnn(input)
                loss = self.criterion(output, target)

                acc = accuracy_VOC2012(output, target)
                losses.update(loss.item(), input.size(0))
                top0.update(acc[0], input.size(0))
#                 if (i+1) % self.print_freq == 0:
#                     enum.set_description('Test: Loss {loss.val:.4f} ({loss.avg:.4f}) '
#                                          'Acc@0 {top0.val:.3f} ({top0.avg:.3f})'
#                                     .format(loss=losses, top0=top0))
        return top0.avg

    def save_checkpoint(self, state, is_best, filename='checkpoint.pth.tar'):
        torch.save(state, filename)
        if is_best:
            shutil.copyfile(filename, 'model_best_{}_{}.pth.tar'
                            .format(self.arch, self.suffix))
    
    def load_checkpoint(self, resume):
        if os.path.isfile(resume):
            print("=> loading checkpoint '{}'".format(resume))
            checkpoint = torch.load(args.resume, map_location=torch.device("cuda:{}".format(self.gpu)))
            self.cnn.load_state_dict(checkpoint['state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {}) as Teacher!"
                  .format(args.resume, checkpoint['epoch']))
            del checkpoint
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))
            return
    
    def fit(self):
        best_acc1 = 0
        acc1 = 0
        acc1_test = 0
        scheduler = lr_scheduler.StepLR(self.optimizer, 
                                        step_size=self.step_size, 
                                        gamma=self.gamma)
        epochs = tqdm.tqdm(range(self.epochs), 
                               desc='Train Acc {:.4f} Test Acc {:.4f} ({:.4f}) Epoch'.format(acc1_test, best_acc1, acc1))
        for epoch in epochs:
            # train for one epoch
            _, acc1_test = self.train(epoch)

            # evaluate on validation set
            acc1 = self.validate()
            
            # remember best acc@1 and save checkpoint
            is_best = acc1 > best_acc1
            best_acc1 = max(acc1, best_acc1)
            if self.save:
                save_dir = 'checkpoint_{}_{}.pth.tar'.format(self.arch, self.suffix)
                if (epoch+1)%self.save_epoch == 0:
                    self.save_checkpoint({
                        'epoch': epoch + 1,
                        'arch': self.arch,
                        'state_dict': self.cnn.state_dict(),
                        'best_acc1': best_acc1,
                        'acc1': acc1,
                        'optimizer' : self.optimizer.state_dict(),
                    }, is_best, save_dir)
                elif is_best:
                    self.save_checkpoint({
                        'epoch': epoch + 1,
                        'arch': self.arch,
                        'state_dict': self.cnn.state_dict(),
                        'best_acc1': best_acc1,
                        'acc1': acc1,
                        'optimizer' : self.optimizer.state_dict(),
                    }, is_best, save_dir)
            epochs.set_description_str('Train Acc {:.4f} Test Acc {:.4f} ({:.4f})'
                                       .format(acc1_test, best_acc1, acc1))
            scheduler.step()
            
    def set_dataset(self, db_train, db_val):
        self.train_loader = DataLoader(db_train, batch_size=self.batch_size, shuffle=True,
                                 num_workers=self.workers, pin_memory=True)
        self.val_loader = DataLoader(db_val, batch_size=self.batch_size, shuffle=False,
                               num_workers=self.workers, pin_memory=True)
    
    def get_score(self, X):
        self.cnn.eval()

        with torch.no_grad():
            if self.gpu is not None:
                input = input.cuda(self.gpu, non_blocking=True)
                target = target.float().unsqueeze(-1).cuda(self.gpu, non_blocking=True)
            output = self.cnn(input)

        return output.cpu()


In [28]:
M = CNN()

In [29]:
db_train = FDDB('train_list.npy', 
                transform=T.Compose([
                    T.RandomHorizontalFlip(),
                    M.img_trans
                ]),
                zero_one=True
               )
db_val = FDDB('val_list.npy', 
              transform=M.img_trans,
              zero_one=True
             )

In [30]:
M.set_dataset(db_train, db_val)

In [31]:
M.fit()









  0%|          | 0/200 [00:00<?, ?it/s]







  0%|          | 1/200 [00:11<39:17, 11.84s/it]







  1%|          | 2/200 [00:23<39:02, 11.83s/it]







  2%|▏         | 3/200 [00:35<38:58, 11.87s/it]







  2%|▏         | 4/200 [00:47<38:57, 11.93s/it]







  2%|▎         | 5/200 [00:59<38:38, 11.89s/it]







  3%|▎         | 6/200 [01:11<38:24, 11.88s/it]







  4%|▎         | 7/200 [01:23<38:16, 11.90s/it]







  4%|▍         | 8/200 [01:35<38:04, 11.90s/it]







  4%|▍         | 9/200 [01:47<37:49, 11.88s/it]

KeyboardInterrupt: 